In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
from six.moves import cPickle
import os
#import multiprocessing

import keras
import keras.backend.tensorflow_backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Dense, Activation
from keras.layers import Flatten
from keras.engine.topology import Input
from keras.optimizers import Adam
from keras import regularizers
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Using TensorFlow backend.


In [0]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.InputLayer(input_shape=(22,24,1)))
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation="relu", 
                 #data_format = "channels_first",
                 kernel_regularizer=regularizers.l2(0.01), 
                 #input_shape=(23,24,1),
                 padding='valid',
                 name="conv2d1"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid', 
                 #data_format = "channels_first",
                 name="maxpool2d1",
                 strides=(2,2)))
#model.add(Dropout(rate=0.1))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), activation="relu", 
                 kernel_regularizer=regularizers.l2(0.01),
                 padding='valid', 
                 #data_format = "channels_first",
                 name="conv2d2"))
#model.add(Conv2D(filters=256, kernel_size=(5, 5), activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Flatten(
    #data_format = "channels_first", 
    name="flatten"))
#model.add(Dense(units=512, kernel_regularizer=regularizers.l2(0.01)))
#model.add(Dropout(rate=0.5))
#model.add(Dense(units=512, kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(units=7, activation="softmax", name="preds"))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#model.load_weights('/gdrive/My Drive/workflow/periodic/code/experiments/cnn/model.h5')
model.load_weights('/gdrive/My Drive/periodic_tf/code/experiments/cnn_22_w o_back_subtraction_float_bins/model.h5')
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d1 (Conv2D)             (None, 18, 20, 16)        416       
_________________________________________________________________
maxpool2d1 (MaxPooling2D)    (None, 9, 10, 16)         0         
_________________________________________________________________
conv2d2 (Conv2D)             (None, 5, 6, 64)          25664     
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
preds (Dense)                (None, 7)                 13447     
Total params: 39,527
Trainable params: 39,527
Non-trainable params: 0
_________________________________________________________________


In [0]:
X_ZTF = np.load("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/X_ZTF.npy")
X_ZTF = X_ZTF.reshape(X_ZTF.shape[0], X_ZTF.shape[1], X_ZTF.shape[2], 1)

In [0]:
X_ZTF.shape

(698, 22, 24, 1)

In [0]:
preds = model.predict_classes(X_ZTF, verbose=1)
preds_proba = model.predict(X_ZTF, verbose=1)

698/698 [==============================] - 0s 48us/sample


In [0]:
np.save("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/preds.npy", preds)
np.save("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/preds_proba.npy", preds_proba)

In [0]:
import numpy as np
preds = np.load("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/preds.npy")
prob = np.load("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/preds_proba.npy")
lengthLC = np.load("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/lengthLC.npy")
lcid = np.load("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/lcid.npy")

In [0]:
import pandas as pd
thres = 0.95
clss = {0:'EW', 1:'EA', 2:'RRab', 3:'RRc', 4:'RRd', 5:'RSCVn', 6:'LPV'}
df = pd.DataFrame(columns=['lcid', 'lengthLC', 'Probability', 'Prediction'])
j = 0
for i in range(prob.shape[0]):
  if np.argmax(prob[i])==preds[i] and np.max(prob[i])>=thres:
    df.loc[j] = [lcid[i]] + [lengthLC[i]] + [prob[i]] + [clss[preds[i]]]
    j = j + 1

In [0]:
df

,lcid,lengthLC,Probability,Prediction
0,10529003000198.3.1.lc,2,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",RRc
1,10529003005233.3.1.lc,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",RRab
2,10529013000652.3.1.lc,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",RRab
3,10529073008030.3.1.lc,2,"[0.0, 0.0, 7.6950143e-16, 1.0, 0.0, 0.0, 0.0]",RRc
4,10529083009474.3.1.lc,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",RRab
5,10529103002361.3.1.lc,2,"[3.784282e-07, 0.0, 8.617454e-29, 0.99999964, ...",RRc
6,10529153007459.3.1.lc,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",RRab
7,10529163003526.3.1.lc,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",RRab
8,10529183006960.3.1.lc,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",RRab
9,10529223006200.3.1.lc,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",RRab


In [0]:
df.to_csv("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/ztf_predictions.csv")

In [0]:
np.where(preds==3)

(array([  0,   9,  14,  28,  31,  35,  38,  40,  42,  49,  50,  51,  52,
         59,  61,  65,  76,  78,  80,  99, 119, 138, 149, 157, 160, 171,
        177, 283, 284, 299, 305, 309, 314, 318, 322, 327, 341, 343, 356,
        359, 362, 364, 372, 382, 384, 387, 389, 413, 420, 452, 459, 478,
        482, 486, 491, 495]),)

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import pandas as pd
import numpy as np

dfr = pd.read_csv("/gdrive/My Drive/periodic_tf/ztf/r_filter_1.1.lcs/ztf_predictions.csv", index_col=0)
dfg = pd.read_csv("/gdrive/My Drive/periodic_tf/ztf/g_filter_2.1.lcs/ztf_predictions.csv", index_col=0)
dfi = pd.read_csv("/gdrive/My Drive/periodic_tf/ztf/i_filter_3.1.lcs/ztf_predictions.csv", index_col=0)

rlcid = np.array(dfr['lcid'])
glcid = np.array(dfg['lcid'])
ilcid = np.array(dfi['lcid'])

rlcid = np.array([rlcid[i].split(".")[0] for i in range(rlcid.shape[0])])
glcid = np.array([glcid[i].split(".")[0] for i in range(glcid.shape[0])])
ilcid = np.array([ilcid[i].split(".")[0] for i in range(ilcid.shape[0])])

In [0]:
np.intersect1d(rlcid, glcid)

array([], dtype='<U14')

In [0]:
np.intersect1d(rlcid, ilcid)

array([], dtype='<U14')

In [0]:
np.intersect1d(glcid, ilcid)

array([], dtype='<U14')

In [0]:
rpred = np.array(dfr['Prediction'])
gpred = np.array(dfg['Prediction'])

In [0]:
np.where(rpred=='LPV')[0].shape

(2506,)